### Importing libraries

In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

Using TensorFlow backend.
/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


### Loading Dataset (sonnets of shakespear)

In [2]:
text=(open("./sonnets.txt").read())
text=text.lower()

### Chart to vector map

In [3]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

### Preparing Data

In [4]:
x = []
y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    x.append([char_to_n[char] for char in sequence])
    y.append(char_to_n[label])

### Data reformation

In [5]:
x_modified = np.reshape(x, (len(x), seq_length, 1))
x_modified = x_modified / float(len(characters))
y_modified = np_utils.to_categorical(y)

In [13]:
y_modified.shape

(119649, 57)

### Model

In [6]:
model = Sequential()
model.add(LSTM(700, input_shape=(x_modified.shape[1], x_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(700))
model.add(Dropout(0.2))
model.add(Dense(y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
model.fit(x_modified, y_modified, epochs=100, batch_size=50)

# model.save('./poemModel.h5')

Epoch 1/100
   450/119649 [..............................] - ETA: 4142s - loss: 3.6071

KeyboardInterrupt: 

### Generating Text

In [ ]:
string_mapped = x[99]
full_string = [n_to_char[value] for value in string_mapped]

for i in range(400):
    xtest = np.reshape(string_mapped,(1,len(string_mapped), 1))
    xtest = xtest / float(len(characters))

    pred_index = np.argmax(model.predict(xtest, verbose=0))
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

### Output

In [ ]:
txt=""
for char in full_string:
    txt = txt+char
txt